In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import re
import nltk
from nltk.corpus import stopwords
import tqdm
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import string
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report


from warnings import filterwarnings
filterwarnings('ignore')

pd.set_option('display.max_colwidth',100)

In [2]:
df = pd.read_csv('./df_file.csv') # encoding='latin-1'
df.head()

,Text,Label
0,Budget to set scene for election\n \n Gordon Brown will seek to put the economy at the centre of...,0
1,Army chiefs in regiments decision\n \n Military chiefs are expected to meet to make a final deci...,0
2,Howard denies split over ID cards\n \n Michael Howard has denied his shadow cabinet was split ov...,0
3,Observers to monitor UK election\n \n Ministers will invite international observers to check the...,0
4,Kilroy names election seat target\n \n Ex-chat show host Robert Kilroy-Silk is to contest the De...,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    2225 non-null   object
 1   Label   2225 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.9+ KB


In [4]:
df.shape

(2225, 2)

In [ ]:
df.Label.unique()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(keep='last', inplace=True)
df.duplicated().sum()

In [ ]:
df.isnull().values.any()

In [ ]:
df.head()

In [ ]:
df.replace({0:"Politics", 1:"Sport", 2:"Technology", 3:"Entertainment", 4:"Business"}, inplace=True)

In [ ]:
df.Label.value_counts()

In [ ]:
import plotly.express as px
import pandas as pd

# Contagem de valores da coluna 'Label'
label_counts = df['Label'].value_counts().reset_index()
label_counts.columns = ['Label', 'Count']

# Gráfico de pizza com buraco (do tipo donut)
fig = px.pie(label_counts,
             values='Count',
             names='Label',
             hole=0.5,
             title='Label distribution of the Text')

fig.show()

In [ ]:
df.head()

In [ ]:
df['Text'][df['Label'] == 'Politics'][0]

In [ ]:
def cleanTweets(text):
    txt = text.lower() #convert all letters to lower case
    txt = re.sub("@[A-Za-z0-9_]+","", txt) #remove mentions
    txt = re.sub(r"http\S+", "", txt) #remove url's https tags
    txt = re.sub(r"www.\S+", "", txt) #remove url's www tags
    txt = re.sub("[^a-z0-9]"," ", txt) #remove non-alphanumeric characters
    txt = re.sub('[\s]+', ' ', txt) #Remove additional white spaces
    return txt

In [ ]:
cleantext = []

for item in df['Text']:
    txt = cleanTweets(item)
    cleantext += [txt]

df['clean_text'] = cleantext
df.head(10)

In [ ]:
df = df.drop(['Text'],axis=1)
df.head()

In [ ]:
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
stop[0:10]

In [ ]:
def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [ ]:
df['clean_text'] = df['clean_text'].apply(remove_stopwords)
df.head()

In [ ]:
df['clean_text'][df['Label'] == 'Politics'][0]

In [ ]:
def create_wordclouds(text, label):
    texts = []
    
    for word_list in df[df['Label']==label]['clean_text'].str.split():
        for word in word_list:
            texts.append(word)
    return texts

In [ ]:
wordcloud_politics = create_wordclouds(df,'Politics')
word_cloud_politics = WordCloud(background_color='white',max_font_size=100).generate(" ".join(wordcloud_politics))
plt.figure(figsize=(12,8))
plt.imshow(word_cloud_politics)
plt.title("Wordcloud for Label - 'Politics'")
plt.axis('off')
plt.show()

In [ ]:
wordcloud_sport = create_wordclouds(df,'Sport')
word_cloud_sport = WordCloud(background_color='black',max_font_size=100).generate(" ".join(wordcloud_sport))
plt.figure(figsize=(12,8))
plt.imshow(word_cloud_sport)
plt.title("Wordcloud for Label - 'Sport'")
plt.axis('off')
plt.show()

In [ ]:
wordcloud_tech = create_wordclouds(df,'Technology')
word_cloud_tech = WordCloud(background_color='gray',max_font_size=80).generate(" ".join(wordcloud_tech))
plt.figure(figsize=(12,8))
plt.imshow(word_cloud_tech)
plt.title("Wordcloud for Label - 'Technology'")
plt.axis('off')
plt.show()

In [ ]:
wordcloud_ent = create_wordclouds(df,'Entertainment')
word_cloud_ent = WordCloud(background_color='gray',max_font_size=80).generate(" ".join(wordcloud_ent))
plt.figure(figsize=(12,8))
plt.imshow(word_cloud_ent)
plt.title("Wordcloud for Label - 'Entertainment'")
plt.axis('off')
plt.show()

In [ ]:
wordcloud_bus = create_wordclouds(df,'Business')
word_cloud_bus = WordCloud(background_color='gray',max_font_size=80).generate(" ".join(wordcloud_bus))
plt.figure(figsize=(12,8))
plt.imshow(word_cloud_bus)
plt.title("Wordcloud for Label - 'Business'")
plt.axis('off')
plt.show()

In [ ]:
df['text_length'] = df['clean_text'].apply(len)
df.head()

In [ ]:
plt.figure(figsize=(10,5))
df['text_length'].plot(bins=50,kind='hist');

In [ ]:
fig,(ax1,ax2,ax3, ax4, ax5) = plt.subplots(nrows=1,ncols=5,figsize=(20,5))

text_len = df[df['Label']=='Politics']['text_length']
ax1.hist(text_len,color='green',bins=30)
ax1.set_title('Politics')

text_len = df[df['Label']=='Sport']['text_length']
ax2.hist(text_len,color='red',bins=30)
ax2.set_title('Sport')

text_len = df[df['Label']=='Technology']['text_length']
ax3.hist(text_len,color='yellow',bins=30)
ax3.set_title('Technology')

text_len = df[df['Label']=='Entertainment']['text_length']
ax4.hist(text_len,color='blue',bins=30)
ax4.set_title('Entertainment')
text_len = df[df['Label']=='Business']['text_length']
ax5.hist(text_len,color='black',bins=30)
ax5.set_title('Business')

fig.suptitle('Character length analysis in Text')
plt.show()

In [ ]:
df.drop(['text_length'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
def get_top_tweet_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
plt.figure(figsize=(10,5))
top_tweet_bigrams=get_top_tweet_bigrams(df['clean_text'])[:10]
x,y=map(list,zip(*top_tweet_bigrams))
sns.barplot(x=y,y=x);

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(df['clean_text'])
vector = vectorizer.transform(df['clean_text'])
print(vector.shape)

In [ ]:
cv = CountVectorizer(stop_words = 'english')
words = cv.fit_transform(df.clean_text)

sum_words = words.sum(axis=0)

words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])

frequency.head(30).plot(x='word', y='freq', kind='bar', figsize=(15, 7), color = 'blue')
plt.title("Most Frequently Occuring Words - Top 30");

In [ ]:
tfidf_converter = TfidfTransformer()
X_tfidf = tfidf_converter.fit_transform(vector).toarray()

In [ ]:
df.head()

In [ ]:
X_tfidf[0:5]

In [ ]:
print(df.Label.value_counts())
sns.countplot(data=df, x='Label');


In [ ]:
X = X_tfidf
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=123)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [ ]:
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)
print("Classification Report Decision Tree:")
print(classification_report(y_test, y_pred_dt))

In [ ]:
cm = confusion_matrix(y_test,y_pred_dt)

import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(cm,
           annot=True,
           fmt='d')
plt.title('prediction result');

In [ ]:
import time
start = time.time()
stop = time.time()
print(f"Training time: {stop - start}s")

In [ ]:
%timeit -n 5 -r 3 dt.predict(X_test)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rf_df = pd.DataFrame(y_pred_rf, columns=['Pred_RandomForestClassifier'])

In [ ]:
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("RF:", accuracy_rf)
print("Classification Report RF:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
cm = confusion_matrix(y_test,y_pred_rf)

import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(cm,
           annot=True,
           fmt='d')
plt.title('prediction result');

In [ ]:
import time
start = time.time()
stop = time.time()
print(f"Training time: {stop - start}s")

In [ ]:
%timeit -n 5 -r 3 rf.predict(X_test)